<div dir='rtl' style='direction:rtl:'>
<font size='5'>
Installation
<font size='3'>
<br>
در این قسمت کتابخانه‌های مورد نیاز نصب و import می‌شوند.
</div>

In [1]:
!pip install transformers

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import re, sys
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<div dir='rtl' style='direction:rtl:'>
<font size='5'>
Data Preprocessing
<font size='3'>
<br>
در این قسمت داده خوانده شده و پیش پردازش روی آن انجام می‌شود. تعداد ستون‌های تگ داده زیاد است اما تنها همان‌هایی که در تمرین از ما خواسته شده‌اند جدا می‌شوند.
</div>

In [3]:
df = pd.read_csv('all-train.csv')
df = df.drop('Unnamed: 0', axis=1)
# select desired columns
df = df[['record_file', 'record_text', 'ABDOMINAL', 'CREATININE','MAJOR-DIABETES']]
df

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,162.xml,Record date: 2068-02-04\n\nASSOCIATED ARTHRITI...,met,not met,not met
1,176.xml,Record date: 2085-04-22\n\n \nThis patient wan...,met,not met,not met
2,189.xml,Record date: 2090-07-07\n\nWillow Gardens Care...,not met,met,met
3,214.xml,Record date: 2096-07-15\n\n\n\nResults01/31/20...,not met,not met,not met
4,200.xml,Record date: 2170-02-17\n\n \n\nReason for Vis...,met,not met,met
...,...,...,...,...,...
197,219.xml,Record date: 2070-01-05\n\n \n \n \n \n \nJanu...,not met,met,met
198,392.xml,"Record date: 2111-09-26\n\nCC: Ear pain, coug...",met,met,met
199,345.xml,Record date: 2072-11-25\n\n ...,not met,met,met
200,184.xml,Record date: 2062-01-29\n\nTRIBAL INTERNAL MED...,not met,met,met


In [4]:
df.shape

(202, 5)

In [5]:
df['record_text'][0]

'Record date: 2068-02-04\n\nASSOCIATED ARTHRITIS SPECIALISTS CENTER            Quijano, Baylee\n                                              2-03-68\n \n \n \nIdentification:  Patient is a 53-year old markedly obese female \ncomplaining of bilateral weight knee pain.  She denies any morning \nstiffness, any jelling phenomena.  She has had no trauma or effusions in \nthe knee.  She had noted on x-ray many years ago that she had had a \nchipped bone in the right knee and some mild osteoarthritis but it has \nbothered her only intermittently until this year.  Last year she did go \nto Briggs Stratton and dropped to 270 lb. at which point her knees felt \nbetter, but currently she is back up to over 300 lb. and at her height \nof 5 ft. 1 in., she is in pain.  She has not visited a physician.  She \nhas no internist who currently checks on her general medical health, but \nshe does report that the occasional Advil she takes does produce some \ndyspepsia.  She has no history of gout, no his

In [6]:
# check if there is any null values
df.isnull().sum()

record_file       0
record_text       0
ABDOMINAL         0
CREATININE        0
MAJOR-DIABETES    0
dtype: int64

In [7]:
le = LabelEncoder()
label_cols = df.columns[2:]
label_cols
for col in label_cols:
  df[col] = le.fit_transform(df[col])

df.head()

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,162.xml,Record date: 2068-02-04\n\nASSOCIATED ARTHRITI...,0,1,1
1,176.xml,Record date: 2085-04-22\n\n \nThis patient wan...,0,1,1
2,189.xml,Record date: 2090-07-07\n\nWillow Gardens Care...,1,0,0
3,214.xml,Record date: 2096-07-15\n\n\n\nResults01/31/20...,1,1,1
4,200.xml,Record date: 2170-02-17\n\n \n\nReason for Vis...,0,1,0


<div dir='rtl' style='direction:rtl:'>
<font size='3'>
<br>
علائم اضافه و همچنین stopwordهای متن از آن حذف می‌شوند.
</div>

In [8]:
# Perform preprocessing to tex
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;*]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z -+.%\':]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
#     text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [9]:
df['record_text'] = df['record_text'].apply(clean_text)
# df['text'] = df['text'].str.replace('\d+', '')
df.head()

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,162.xml,record date: 20680204associated arthritis spec...,0,1,1
1,176.xml,record date: 20850422 patient wants transfer c...,0,1,1
2,189.xml,record date: 20900707willow gardens care colum...,1,0,0
3,214.xml,record date: 20960715results01 31 2096 glupoc ...,1,1,1
4,200.xml,record date: 21700217 reason visit:chemotherap...,0,1,0


In [10]:
df['record_text'][0]

'record date: 20680204associated arthritis specialists center quijano baylee 20368 identification: patient 53year old markedly obese female complaining bilateral weight knee pain. denies morning stiffness jelling phenomena. trauma effusions knee. noted xray many years ago chipped bone right knee mild osteoarthritis bothered intermittently year. last year go briggs stratton dropped 270 lb. point knees felt better currently back 300 lb. height 5 ft. 1 in. pain. visited physician. internist currently checks general medical health report occasional advil takes produce dyspepsia. history gout history psoriasis history pseudogout rash alopecia headaches shoulder hip discomfort. complain back pain difficult walking 50 yards knee pain. allergies: none. medications: occasional advil. past surgical history: cholecystectomy 2064. two csections. past medical history: negative hypertension diabetes cancer stroke tuberculosis hepatitis asthma heart disease. family history: mother alive well medical 

<div dir='rtl' style='direction:rtl:'>
<font size='3'>
<br>
از آنجایی که متن ما طولانی است و مدل BERT محدودیت روی تعداد کلمات ورودی دارد، متن را به قسمت‌های ۲۰۰ کلمه‌ای می‌شکنیم که ۵۰ کلمه با هم overlap دارند.
</div>

In [11]:
# Processing text for BERT model, spliting text into chuncks of 200 words with ith 50 words overlapped.
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//150 >0:
        n = len(text1.split())//150
    else:
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:200]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_parcial))
    return l_total

In [12]:
df['record_text_split'] = df['record_text'].apply(get_split)
df.head()

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES,record_text_split
0,162.xml,record date: 20680204associated arthritis spec...,0,1,1,[record date: 20680204associated arthritis spe...
1,176.xml,record date: 20850422 patient wants transfer c...,0,1,1,[record date: 20850422 patient wants transfer ...
2,189.xml,record date: 20900707willow gardens care colum...,1,0,0,[record date: 20900707willow gardens care colu...
3,214.xml,record date: 20960715results01 31 2096 glupoc ...,1,1,1,[record date: 20960715results01 31 2096 glupoc...
4,200.xml,record date: 21700217 reason visit:chemotherap...,0,1,0,[record date: 21700217 reason visit:chemothera...


In [13]:
df['record_text_split'][0]

['record date: 20680204associated arthritis specialists center quijano baylee 20368 identification: patient 53year old markedly obese female complaining bilateral weight knee pain. denies morning stiffness jelling phenomena. trauma effusions knee. noted xray many years ago chipped bone right knee mild osteoarthritis bothered intermittently year. last year go briggs stratton dropped 270 lb. point knees felt better currently back 300 lb. height 5 ft. 1 in. pain. visited physician. internist currently checks general medical health report occasional advil takes produce dyspepsia. history gout history psoriasis history pseudogout rash alopecia headaches shoulder hip discomfort. complain back pain difficult walking 50 yards knee pain. allergies: none. medications: occasional advil. past surgical history: cholecystectomy 2064. two csections. past medical history: negative hypertension diabetes cancer stroke tuberculosis hepatitis asthma heart disease. family history: mother alive well medical

<div dir='rtl' style='direction:rtl:'>
<font size='3'>
<br>
۸۰ درصد داده به عنوان داده‌ی train، ۱۰ درصد داده‌ی validatoin و 10 درصد نیز به عنوان داده‌ی تست جدا می‌شود.
</div>

In [14]:
train, test = train_test_split(df, test_size=0.1, random_state=35)
test.reset_index(drop=True, inplace=True)
test.head(2)

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES,record_text_split
0,333.xml,record date: 21220223 source note mch emergenc...,0,1,1,[record date: 21220223 source note mch emergen...
1,319.xml,record date: 20850821mima 5z36 santa fe road s...,1,1,0,[record date: 20850821mima 5z36 santa fe road ...


In [15]:
train, val = train_test_split(train, test_size=0.111, random_state=35)
val.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)
train.head(2)

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES,record_text_split
0,362.xml,record date: 20700818 kekela emergency dept vi...,1,1,0,[record date: 20700818 kekela emergency dept v...
1,188.xml,record date: 20660305internal medicine associa...,1,1,1,[record date: 20660305internal medicine associ...


In [16]:
print("train shape:", train.shape)
print("val shape:", val.shape)
print("test shape:", test.shape)

train shape: (160, 6)
val shape: (21, 6)
test shape: (21, 6)


In [17]:
def use_splits(df):
  record_file_l = []
  data_l = []
  label_l_1 = []
  label_l_2 = []
  label_l_3 = []
  index_l =[]
  for idx,row in df.iterrows():
      for l in row['record_text_split']:
          record_file_l.append(row['record_file'])
          data_l.append(l)
          label_l_1.append(row['ABDOMINAL'])
          label_l_2.append(row['CREATININE'])
          label_l_3.append(row['MAJOR-DIABETES'])
          index_l.append(idx)

  data_df = pd.DataFrame({'record_file':record_file_l, 'record_text':data_l, 'ABDOMINAL':label_l_1, 'CREATININE':label_l_2, 'MAJOR-DIABETES':label_l_3})
  return data_df, index_l

train_df, train_idx = use_splits(train)
val_df, val_idx = use_splits(val)
test_df, test_idx = use_splits(test)

train_df

,record_file,record_text,ABDOMINAL,CREATININE,MAJOR-DIABETES
0,362.xml,record date: 20700818 kekela emergency dept vi...,1,1,0
1,362.xml,nontender normal bowel sounds epigastric pain ...,1,1,0
2,362.xml,attending physician record date: 20720408 keke...,1,1,0
3,362.xml,2+ without bruits. chest: clear auscultation p...,1,1,0
4,362.xml,57yrold female w cp syndrome crfs dm htn docum...,1,1,0
...,...,...,...,...,...
1878,378.xml,l 100108 mmol lplasma carbon dioxide 27.1 23.0...,0,1,0
1879,378.xml,1. significant change compared prior ct brain0...,0,1,0
1880,378.xml,foley nowresend u culture 12 days antibioticsh...,0,1,0
1881,378.xml,represents ed today husband primary care giver...,0,1,0


## Fine tuning the BERT model

In [18]:
target_list = ['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES']

In [19]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 4
LEARNING_RATE = 1e-05

In [20]:
from transformers import BertTokenizer, BertModel

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
!pip install -U sentence-transformers -q

In [23]:
from sentence_transformers import SentenceTransformer
BioBert = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')

In [24]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.text = df['record_text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'record_file': self.df['record_file'][index],
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [25]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)


In [26]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [28]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

<div dir='rtl' style='direction:rtl:'>
<font size='3'>
<br>
در این قسمت مدلی تعریف می‌شود که یک لایه‌ی linear سه نرونه بر روی مدل BERT دارد تا عمل multi-label classification را انجام دهد.
</div>

In [29]:
class BERTClass(torch.nn.Module):
    def __init__(self, biobert = False):
        super(BERTClass, self).__init__()
        if not biobert:
          self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        else:
          self.bert_model = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear1 = torch.nn.Linear(768, 300)
        self.linear2 = torch.nn.Linear(300, 150)
        self.dropout2 = torch.nn.Dropout(0.3)
        self.linear3 = torch.nn.Linear(150, 25)
        self.linear4 = torch.nn.Linear(25, 3)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear1(output_dropout)
        output = self.linear2(output)
        output=self.dropout2(output)
        output = self.linear3(output)
        output = self.linear4(output)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [30]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [31]:
val_targets=[]
val_outputs=[]

In [32]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      # save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [33]:
ckpt_path = "/content/curr_ckpt"
best_model_path = "/content/best_model.pt"

<div dir='rtl' style='direction:rtl:'>
<font size='3'>
<br>
با توجه به ریسورس‌های پردازشی موجود مدل به تعدادی زیر ۵ ایپاک train می‌شود.
</div>

In [34]:
EPOCHS = 15
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.011653 	Average Validation Loss: 0.075639
Validation loss decreased (inf --> 0.075639).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.011546 	Average Validation Loss: 0.075545
Validation loss decreased (0.075639 --> 0.075545).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

## Testing the model

In [35]:
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)

In [36]:
# testing
final_outputs = []
true_outputs = []
record_files = []
model.eval()
with torch.no_grad():
    for idx in range(len(test_dataset)):
        data = test_dataset[idx]
        input_ids = data['input_ids'].unsqueeze(0).to(device, dtype = torch.long)
        attention_mask = data['attention_mask'].unsqueeze(0).to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].unsqueeze(0).to(device, dtype = torch.long)
        targets = data['targets'].unsqueeze(0)

        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
        final_output = [ 1 if x>=0.5 else 0 for x in final_output[0]]
        final_outputs.append(final_output)
        targets = [int(x) for x in targets.tolist()[0]]
        true_outputs.append(targets)
        record_files.append(data['record_file'])


In [37]:
ABDOMINAL = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}
CREATININE = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}
MAJOR_DIABETES = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}

def calc(wanted_dic, pred_y, true_y):
  if pred_y == true_y == 1:
    wanted_dic['TP'] += 1
  elif pred_y == true_y == 0:
    wanted_dic['TN'] += 1
  elif pred_y != true_y and pred_y == 1:
    wanted_dic['FP'] += 1
  elif pred_y != true_y and pred_y == 0:
    wanted_dic['FN'] += 1
  return wanted_dic


record_file_prev = ''
for i in range(len(final_outputs)):
  pred_y = final_outputs[i]
  true_y = true_outputs[i]
  record_file = record_files[i]
  calc(ABDOMINAL,pred_y[0],true_y[0])
  calc(CREATININE,pred_y[1],true_y[1])
  calc(MAJOR_DIABETES,pred_y[2],true_y[2])

print('ABDOMINAL', ABDOMINAL)
print('CREATININE', CREATININE)
print('MAJOR_DIABETES', MAJOR_DIABETES)

ABDOMINAL {'TP': 128, 'TN': 43, 'FP': 51, 'FN': 55}
CREATININE {'TP': 174, 'TN': 19, 'FP': 42, 'FN': 42}
MAJOR_DIABETES {'TP': 46, 'TN': 98, 'FP': 76, 'FN': 57}


In [41]:
def calc_metrics(dic):
  tp = dic['TP']
  tn = dic['TN']
  fp = dic['FP']
  fn = dic['FN']
  acc = (tp + tn) / (tp + tn + fp + fn)
  recall = tp / (tp + fn)
  precision = tp / (tp + fp)

  return acc, recall, precision

print("ABDOMINAL:", calc_metrics(ABDOMINAL))
print("CREATININE:", calc_metrics(CREATININE))
print("MAJOR_DIABETES:", calc_metrics(MAJOR_DIABETES))

ABDOMINAL: (0.6173285198555957, 0.6994535519125683, 0.7150837988826816)
CREATININE: (0.6967509025270758, 0.8055555555555556, 0.8055555555555556)
MAJOR_DIABETES: (0.51985559566787, 0.44660194174757284, 0.3770491803278688)


### Interpretibility

In [39]:
!pip install shap

In [40]:
import shap

# explain all the predictions in the test set
explainer = shap.KernelExplainer(nn.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

AttributeError: ignored